In [1]:
from genai import Credentials, Client
from genai.schema import TextGenerationParameters, TextGenerationReturnOptions
from tqdm import tqdm
import os
import re
import jsonschema
from jsonschema import TypeChecker
import json

In [89]:
text = '''System:
You are an intelligent AI programming assistant, utilizing a Granite code language model developed by IBM. Your primary function is to assist users in code explanation, code generation and other software engineering tasks. You MUST follow these guidelines: - Your responses must be factual. Do not assume the answer is "yes" when you do not know, and DO NOT SHARE FALSE INFORMATION. - You should give concise answers. You should follow the instruction and provide the answer in the specified format and DO NOT SHARE FALSE INFORMATION.

Question:
You are given a JSON format schema and a JSON sample. Validate if JSON sample adheres to JSON schema. While validating note that if any field defined as an empty object `{}` in the schema, then it means any value is allowed for that.

Select answer from two options "yes" or "no".
If JSON sample adheres to JSON schema for every constraint answer "yes" else answer "no".Below are examples demonstrating the same.

JSON schema:
{
    "type": "array",
    "contains": {
        "type": "boolean"
    },
    "minContains": 0
}

JSON sample:
[true, true, false]

Answer:
```
yes
```

JSON schema:
{
    "type": "array",
    "contains": {
        "type": "string"
    }
}

JSON sample:
["abcd", "bghi", 13.63]

Answer:
```
no
```

JSON schema:
{
    "type": "array",
    "contains": {
        "type": "number",
        "exclusiveMaximum": 67.39270466877734
    },
    "minContains": 0,
    "maxContains": 100
}

JSON sample:
[8.0, 31.0, 25.0, 30.0, 50.0, 44.0, 4.0, 43.0, 2.0, 62.0, 49.0, 14.0, 65.0, 34.0, 24.0, 0.0, 48.0, 61.0, 66.0, 50.0, 61.0, 0.0, 59.0, 55.0, 50.0, 0.0, 45.0, 2.0, 0.0, 49.0, 24.0, 5.0, 16.0, 65.0, 21.0, 53.0, 35.0, 48.0, 36.0, 12.0, 46.0, 65.0, 21.0, 60.0, 55.0, 2.0, 58.0, 48.0, 33.0, 35.0, 25.0, 17.0, 3.0, 25.0, 33.0, 66.0, 34.0, 2.0, 52.0, 50.0]

Answer:
```
yes
```'''

In [91]:
answers = re.findall(r'Answer:\s*```([\s\S]*?)```', text)
# Print the extracted answers
# for answer in answers:
#     print(answer.strip())

'\nyes\n'

In [2]:
base_path = "./constrain-data-gen-eval/data_generator/data_generator/"
xml_path = "xml_dataset_6_march/"
yaml_path = "yaml_dataset_6_march/"
json_path = "json_schema_dataset_28_feb/"
python_path = "python_schema_data_4th_june/"
task_2_path = "Task_2/json/"

xml_files = os.listdir(base_path + xml_path)
yaml_files = os.listdir(base_path + yaml_path)
json_files = os.listdir(base_path + json_path)
python_files = os.listdir(base_path + python_path)
task_2_files = os.listdir(base_path + task_2_path)

In [3]:
import os

output_base_path = "constrain-data-gen-eval/data_generator/data_generator/Task_2/x_to_JSON/JSON_2_JSON/granite/8B/few_shot/2_shot/"
output_files = os.listdir(output_base_path)

# output_base_path_2 = "constrain-data-gen-eval/data_generator/data_generator/Task_2/x_to_JSON/JSON_2_JSON/granite/20B/zero_shot/prompt_1/"
# output_files_2 = os.listdir(output_base_path_2)

# output_base_path_3 = "constrain-data-gen-eval/data_generator/data_generator/Task_2/x_to_JSON/JSON_2_JSON/granite/34B/zero_shot/prompt_1/"
# output_files_3 = os.listdir(output_base_path_3)

In [3]:
import re

def extract_text_between_sample_tags(text, sample_tag=False):
#     # Define the regular expression pattern to match text between [SAMPLE] and [/SAMPLE]
#     if sample_tag:
#         pattern = r'\[SAMPLE\]\n(.*?)\[/SAMPLE\]'
#     else:
# #         pattern = r'```(.*?)```'
# #         pattern = r'JSON sample:\n```json\n(.*?)```'
#         pattern = r'''answer "yes" else answer "no".\n\nAnswer:\n```(.*?)```'''


    
#     # Use re.findall to find all matches
#     matches = re.findall(pattern, text, re.DOTALL)
    
#     if len(matches)==0 and sample_tag==False:
# #         pattern = r'''JSON sample:\n```(.*?)```'''
#         pattern = r'''answer "yes" else answer "no".\n\nAnswer:\n```(.*?)```'''

#         matches = re.findall(pattern, text, re.DOTALL)
        
#     return matches
    answers = re.findall(r'Answer:\s*```([\s\S]*?)```', text)
    if len(answers) > 0:
        return answers[0].strip().lower()
    else:
        return -1

In [69]:
path = "constrain-data-gen-eval/data_generator/data_generator/Task_2/x_to_YAML/NL_2_YAML/granite/"

In [70]:
def find_common_files(path):
    
    files_all = []
    for key in ["8B", "20B", "34B"]:
#     for key in ["8B", "34B"]:
        temp_files = []
        output_base_path = path + key + "/zero_shot/prompt_1_new/"
        output_files = os.listdir(output_base_path)
        
        for file in tqdm(output_files):
            f_sample = open(base_path+task_2_path+file.split(".")[0] + ".json", "r")
            sample_text = f_sample.read()
            schema_file = "_".join(file.split("_")[1:])
            f_schema = open(base_path+json_path+schema_file.split(".")[0] + ".json", "r")
            schema_text = f_schema.read()
            flag = 0
            answer_file = open(output_base_path+file, "r")
            answer_text = answer_file.read()
            extracted_texts = extract_text_between_sample_tags(answer_text, sample_tag=False)
            if(extracted_texts == -1):
                continue
#             for i, extracted_text in enumerate(extracted_texts, 1):
            answer = extracted_texts.strip()
            if answer.startswith("yes"):
                        temp_files.append(file)
            elif answer.startswith("no"):
                        temp_files.append(file)
#                     else:
#                         print(file)
#                         print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
        files_all.append(temp_files)
    
    granite_8B_20B = set(files_all[0]).intersection(files_all[1])
    granite_8B_20B_34B = granite_8B_20B.intersection(files_all[2])
    return(granite_8B_20B)


In [71]:
common_files = find_common_files(path)


100%|██████████████████████████████████████| 2990/2990 [00:10<00:00, 296.82it/s]


In [48]:
len(common_files)

2721

In [12]:
granite_8B_answers = {}
granite_20B_answers = {}
granite_34B_answers = {}

In [72]:
all_answers = []
ground_truth = []
incomplete_files = []
incomplete_json = 0

f1_dic = {}
for key in ["8B", "20B", "34B"]:
# for key in ["8B", "34B"]:
    
    output_base_path = path + key + "/zero_shot/prompt_1_new/"
    output_files = os.listdir(output_base_path)
    temp_answers = []
    for file in tqdm(common_files):
    
        f_sample = open(base_path+task_2_path+file.split(".")[0] + ".json", "r")
        sample_text = f_sample.read()
        schema_file = "_".join(file.split("_")[1:])
        f_schema = open(base_path+json_path+schema_file.split(".")[0] + ".json", "r")
        schema_text = f_schema.read()
        flag = 0
        answer_file = open(output_base_path+file, "r")
        answer_text = answer_file.read()
        extracted_texts = extract_text_between_sample_tags(answer_text, sample_tag=False)
    

        if extracted_texts == -1:
            incomplete_files.append(file)
            incomplete_json += 1
        else:
#             for i, extracted_text in enumerate(extracted_texts, 1):
                answer = extracted_texts.strip()
                if answer.startswith("yes"):
                    temp_answers.append(1)
                    flag = 1
                elif answer.startswith("no"):
                    temp_answers.append(0)
                    flag = 1
#                 else:
#                     print(answer_text)
#                     print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    all_answers.append(temp_answers)


100%|█████████████████████████████████████| 2298/2298 [00:00<00:00, 5780.40it/s]


In [73]:
ground_truth = []

for file in tqdm(common_files):
    
        f_sample = open(base_path+task_2_path+file.split(".")[0] + ".json", "r")
        sample_text = f_sample.read()
        schema_file = "_".join(file.split("_")[1:])
        f_schema = open(base_path+json_path+schema_file.split(".")[0] + ".json", "r")
        schema_text = f_schema.read()
        schema = json.loads(schema_text)
        sample = json.loads(sample_text)
        validator = jsonschema.Draft7Validator(schema)
        i = 0
        for error in validator.iter_errors(sample):
            i += 1
                    
        if i==0:
            ground_truth.append(1)
        else:
            ground_truth.append(0)

100%|█████████████████████████████████████| 2298/2298 [00:00<00:00, 5402.37it/s]


In [51]:
len(all_answers[2])

2348

In [16]:
print(len([i for i in all_answers[0] if i==0]))
print(len([i for i in all_answers[0] if i==1]))

0
2978


In [14]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(ground_truth, all_answers[0]).ravel()

In [15]:
print(tn, fp, fn, tp)

0 1329 0 1649


In [17]:
pre = tp/(tp+fp)
rec = tp/(tp+fn)
f1 = 2*pre*rec/(pre+rec)
print(pre, rec, f1)

0.8488764044943821 0.9967018469656992 0.9168689320388349


In [9]:
from sklearn.metrics import classification_report
x = classification_report(ground_truth, all_answers[0], output_dict=1)

/u/sameer/anaconda3/lib/python3.11/site-packages/sklearn/utils/_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


In [25]:
x['macro avg']['precision'], x['macro avg']['recall'], x['macro avg']['f1-score']

(0.7501759475924782, 0.7227035469885983, 0.7245578319606901)

In [77]:
print(classification_report(ground_truth[:1714], all_answers[2]))

              precision    recall  f1-score   support

           0       0.39      0.55      0.46       703
           1       0.57      0.40      0.47      1011

    accuracy                           0.46      1714
   macro avg       0.48      0.48      0.46      1714
weighted avg       0.49      0.46      0.47      1714



In [57]:
from sklearn.metrics import f1_score
f1_score(ground_truth, all_answers, average='macro')

0.5853513196963003

In [47]:
print(len([i for i in ground_truth if i==0]))
print(len([i for i in ground_truth if i==1]))
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print(len([i for i in all_answers if i==0]))
print(len([i for i in all_answers if i==1]))

269
1516
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
22
1763


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-8b-code-instruct")
model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-8b-code-instruct", device_map="auto", cache_dir = "/dccstor/ai4code-summ/benchmark-paper", torch_dtype=torch.float16)


/u/sameer/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
input_repr = "JSON"
output_repr = "JSON"
input_path = "json_schema_dataset_28_feb/"

In [22]:
import torch
preds = []
for file in tqdm(task_2_files[:10]):
    
    f_sample = open(base_path+task_2_path+file, "r")
    sample = f_sample.read()
    schema_file = "_".join(file.split("_")[1:])
    if input_repr != "Python":
        f_schema = open(base_path+input_path+schema_file.split(".")[0] + "." + input_repr.lower(), "r")
    else:
        f_schema = open(base_path+input_path+schema_file.split(".")[0] + ".py", "r")
    schema = f_schema.read()

#     prompt_1 = '''Question:\nYou are given a XML format schema and a XML sample. Validate if XML sample adhere to XML schema.\n\nXML schema:\n''' + schema.strip() + "\n\nXML sample:\n" + sample.strip() + '''\n\nSelect answer from two options "yes" or "no".\n\nIf XML sample adheres to XML schema for every constraints answer "yes" else answer "no".\n\nIf Your answer is "no", list down attributes in XML sample that do not follow schema constraints.\n\nAnswer:\n```\n'''
    scores = []
    options = ['''yes\n```''', '''no\n```''']
    for option in options:
        prompt = '''Question:\nYou are given a {input_repr} format schema and a {output_repr} sample. Validate if {output_repr} sample adheres to {input_repr} schema.\nWhile validating note that if any field defined as an empty object `{{}}` in the schema, then it means any value is allowed for that.\n\n{input_repr} schema:\n{schema}\n\n{output_repr} sample:\n{sample}\n\nSelect answer from two options "yes" or "no".\nIf {output_repr} sample adheres to {input_repr} schema for every constraint answer "yes" else answer "no".\n\nAnswer:\n```\n'''.format(input_repr=input_repr, output_repr=output_repr, schema=schema.strip(), sample=sample.strip())

        prompt = prompt + option
        tokenized_label = tokenizer(prompt,return_tensors='pt')
        outputs = model(**tokenized_label, labels=tokenized_label.input_ids)
#     top_indexes = outputs.logits.topk(10).indices.flatten().tolist()[-30:]
        scores.append(outputs.loss.item()) #+token_bias_probs[cur_label_idx].item()
    preds.append(torch.argmin(torch.tensor(scores)))

100%|███████████████████████████████████████████| 10/10 [00:14<00:00,  1.45s/it]


In [23]:
print(preds)

[tensor(1), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0)]
